In [200]:
DATABASE_HOST="172.22.0.16"
# DATABASE_HOST="172.20.0.3"
DATABASE_DATABASE="fabricexplorer"
DATABASE_USERNAME="hppoc"
DATABASE_PASSWORD="password"
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import bamboolib as bam
# import hiplot as hip
import matplotlib.pyplot as ptlib
import plotly.express as px
import json
# import js2py

# to get the ips of docker containers
# list all user defined tables and schemas
# "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"

#initializing resources
engine = create_engine('postgresql://'+DATABASE_USERNAME+':'+DATABASE_PASSWORD+'@'+DATABASE_HOST+':5432/'+DATABASE_DATABASE)
JSLONG=js2py.require('long')

txQuery = "SELECT * FROM transactions"
txDf = pd.read_sql(txQuery,con=engine)
# txDf.to_csv('backup.csv')
txDf=txDf[txDf['read_set'].isna()==False]
txDf=txDf[txDf['chaincodename'].str.contains('_lifecycle')==False]
txDf=txDf[txDf['type'].str.contains('CONFIG')==False]
txDf=txDf.drop(columns=["txhash","chaincode_id","endorser_msp_id","type","createdt","channel_genesis_hash","envelope_signature","creator_id_bytes", "creator_nonce", "payload_extension","tx_response","payload_proposal_hash","endorser_id_bytes","endorser_signature", "network_name"])
txDf

    id  blockid chaincodename  status    creator_msp_id  \
0    6        6        simple   200.0  org0-example-com   
7   13        7        simple   200.0  org0-example-com   
8   14        8        simple   200.0  org0-example-com   
9   15        9        simple   200.0  org0-example-com   
10  16       10        simple   200.0  org0-example-com   
11  17       11        simple   200.0  org0-example-com   
12  18       12        simple   200.0  org0-example-com   
13  19       13        simple   200.0  org0-example-com   

                                             read_set  \
0   [{'chaincode': '_lifecycle', 'set': [{'key': '...   
7   [{'chaincode': '_lifecycle', 'set': [{'key': '...   
8   [{'chaincode': '_lifecycle', 'set': [{'key': '...   
9   [{'chaincode': '_lifecycle', 'set': [{'key': '...   
10  [{'chaincode': '_lifecycle', 'set': [{'key': '...   
11  [{'chaincode': '_lifecycle', 'set': [{'key': '...   
12  [{'chaincode': '_lifecycle', 'set': [{'key': '...   
13  [{'chain

In [201]:
read_set=txDf["read_set"]
write_set=txDf["write_set"]
for item in read_set:
    item.pop(0)
for item in write_set:
    item.pop(0)

In [202]:
txDf['read_set']=read_set
txDf['write_set']=write_set
txDf = txDf.explode('read_set')
txDf = txDf.explode('write_set')
txDf = txDf.loc[(txDf['read_set'].astype('string').str.contains('version', case=False, regex=False, na=False))]
txDf

    id  blockid chaincodename  status    creator_msp_id  \
7   13        7        simple   200.0  org0-example-com   
8   14        8        simple   200.0  org0-example-com   
9   15        9        simple   200.0  org0-example-com   
10  16       10        simple   200.0  org0-example-com   
11  17       11        simple   200.0  org0-example-com   
12  18       12        simple   200.0  org0-example-com   
13  19       13        simple   200.0  org0-example-com   

                                             read_set  \
7   {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   
8   {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   
9   {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   
10  {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   
11  {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   
12  {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   
13  {'chaincode': 'simple', 'set': [{'key': ' 􏿿ini...   

                                            write_set validation_code 

In [203]:
txDf['r_chaincode'] =txDf['read_set'].apply(lambda x: x['chaincode'])
txDf['rs'] =txDf['read_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['chaincodename','read_set'])
txDf['w_chaincode'] =txDf['write_set'].apply(lambda x: x['chaincode'])
txDf['ws'] =txDf['write_set'].apply(lambda x: x['set'])
txDf = txDf.drop(columns=['write_set'])
txDf

    id  blockid  status    creator_msp_id validation_code  \
7   13        7   200.0  org0-example-com           VALID   
8   14        8   200.0  org0-example-com           VALID   
9   15        9   200.0  org0-example-com           VALID   
10  16       10   200.0  org0-example-com           VALID   
11  17       11   200.0  org0-example-com           VALID   
12  18       12   200.0  org0-example-com           VALID   
13  19       13   200.0  org0-example-com           VALID   

   chaincode_proposal_input r_chaincode  \
7        690760065,61,62,35      simple   
8        690760065,61,62,35      simple   
9        690760065,62,61,35      simple   
10       690760065,62,61,35      simple   
11       690760065,62,61,35      simple   
12       690760065,62,61,35      simple   
13       690760065,62,61,35      simple   

                                                   rs w_chaincode  \
7   [{'key': ' 􏿿initialized', 'version': {'block_n...      simple   
8   [{'key': ' 􏿿initialized'

In [204]:
def classifyWrite(ws):
    if str(ws)=='nan':
        return np.nan
    elif ws['is_delete']:
        return 'DELETE'
    else:
        return 'WRITE'
txDfr = txDf.drop(columns=['w_chaincode','ws']).explode('rs')
txDfr = txDfr.rename(columns={'r_chaincode': 'chaincode','rs':'key'})
txDfr['access_type'] = txDfr['key'].apply(lambda x: 'READ')
txDfr['version_block'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['block_num']['low'])
txDfr['version_tx'] = txDfr['key'].apply(lambda x: np.nan if str(x) == 'nan' else x['version']['tx_num']['low'])
txDfw = txDf.drop(columns=['r_chaincode','rs']).explode('ws')
txDfw = txDfw.rename(columns={'w_chaincode': 'chaincode','ws':'key'})
txDfw['access_type'] = txDfw['key'].apply(lambda x: classifyWrite(x))
txDf = txDfr.append(txDfw).reset_index(drop=True).rename(columns={'id':'txid'})
txDf['key']=txDf['key'].apply(lambda x:np.nan if str(x) == 'nan' else x['key'])
txDf

    txid  blockid  status    creator_msp_id validation_code  \
0     13        7   200.0  org0-example-com           VALID   
1     13        7   200.0  org0-example-com           VALID   
2     13        7   200.0  org0-example-com           VALID   
3     14        8   200.0  org0-example-com           VALID   
4     14        8   200.0  org0-example-com           VALID   
5     14        8   200.0  org0-example-com           VALID   
6     15        9   200.0  org0-example-com           VALID   
7     15        9   200.0  org0-example-com           VALID   
8     15        9   200.0  org0-example-com           VALID   
9     16       10   200.0  org0-example-com           VALID   
10    16       10   200.0  org0-example-com           VALID   
11    16       10   200.0  org0-example-com           VALID   
12    17       11   200.0  org0-example-com           VALID   
13    17       11   200.0  org0-example-com           VALID   
14    17       11   200.0  org0-example-com           V

In [205]:
txDfr=txDf[txDf['access_type']=='READ']
labels={"x":"Read keys","y":"Number of reads"}
fig = px.bar(x=txDfr['key'].unique(),y=txDfr['key'].value_counts(),labels=labels)
fig

In [206]:
txDfw=txDf[txDf['access_type']=='WRITE']
labels={"x":"Written keys","y":"Number of writes"}
fig = px.bar(x=txDfw['key'].unique(),y=txDfw['key'].value_counts(),labels=labels)
fig

In [207]:
labels={"x":"Validation code","y":"Qty"}
fig = px.bar(x=txDfr['validation_code'].unique(),y=txDfr['validation_code'].value_counts(),labels=labels)
fig

In [208]:
labels={"x":"Chaincode","y":"Key reads"}
fig = px.bar(x=txDfr['chaincode'].unique(),y=txDfr['chaincode'].value_counts(),labels=labels)
fig

In [209]:
labels={"x":"Chaincode","y":"Key writes"}
fig = px.bar(x=txDfw['chaincode'].unique(),y=txDfw['chaincode'].value_counts(),labels=labels)
fig

In [210]:
query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'information_schema' AND schemaname != 'pg_catalog';"
# query = "select * from peer"
idkdf = pd.read_sql(query,con=engine)
idkdf

  schemaname           tablename tableowner tablespace  hasindexes  hasrules  \
0     public              blocks      hppoc       None        True     False   
1     public          chaincodes      hppoc       None        True     False   
2     public  peer_ref_chaincode      hppoc       None        True     False   
3     public             channel      hppoc       None        True     False   
4     public                peer      hppoc       None        True     False   
5     public    peer_ref_channel      hppoc       None        True     False   
6     public             orderer      hppoc       None        True     False   
7     public        transactions      hppoc       None        True     False   
8     public               users      hppoc       None        True     False   
9     public          write_lock      hppoc       None        True     False   

   hastriggers  rowsecurity  
0        False        False  
1        False        False  
2        False        False  

In [241]:
import binascii
import curses
# binascii.hexlify('a'.encode('utf8'))
# "e".encode('utf8').hex()
# bytes.fromhex('69').decode()
# bytes.fromhex('690760065').decode('utf8')
# binascii.unhexlify('6900760065').decode('utf8')
print(str(b"690760065",'utf8'))

690760065
